# L2 — Memory / Chat History (LangChain v1)

# Setup

This notebook uses **OpenAI (Python SDK v2) + LangChain v1**.

## Prereqs
1. Set your API key in the environment:

```bash
export OPENAI_API_KEY="..."
```

2. Restart the kernel after setting env vars.


In [ ]:
import os

# Make sure your key is set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running."

MODEL = "gpt-5-mini"


LangChain v1 encourages *explicit* message history.

We'll use `RunnableWithMessageHistory` to persist a conversation in memory.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model=MODEL)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant who remembers the conversation."),
    MessagesPlaceholder("history"),
    ("user", "{input}")
])

base_chain = prompt | llm

# One store per session_id
_history_store = {}

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in _history_store:
        _history_store[session_id] = InMemoryChatMessageHistory()
    return _history_store[session_id]

chain = RunnableWithMessageHistory(
    base_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history",
)

session = {"configurable": {"session_id": "demo"}}

print(chain.invoke({"input": "Hi! My name is Ada."}, config=session).content)
print(chain.invoke({"input": "What's my name?"}, config=session).content)


## Resetting memory

In [ ]:
_history_store.pop("demo", None)
print("Cleared.")